<a href="https://colab.research.google.com/github/jenieto/computer-vision/blob/use-opencv-imshow/computer-vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Trabajo M0

In [0]:
# Montamos Google Drive
from google.colab import drive
drive.mount('/content/drive/')
!unzip -o "/content/drive/My Drive/Datasets/computer-vision-M2.zip" -d /content/

In [0]:
# Importamos librerias
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
import os 
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split

In [0]:
# Definimos variables
csv_path = '/content/computer-vision-M2/anotaciones_itemsEvaluables_v3.csv'
images_path = '/content/computer-vision-M2/dataset1'
figure = 'a1'
image_shape = (512, 512, 1)
test_dataset_size = 0.25

In [0]:
# Funciones para leer los datos
def filter_figure(data, figure):
  filter = data['figure'] == figure
  return data[filter]
  
def read_csv_data(figure=None):
  data = pd.read_csv(csv_path, sep=';', names=['dir', 'image', 'figure', 'coords'], index_col=False, skipinitialspace=True)
  for key in data.keys():
    data[key] = data[key].apply(str.replace, args=(' ', ''))
  if figure is not None:
    data = filter_figure(data, figure)
  return data

In [0]:
# Leemos los datos
data = read_csv_data(figure)
data

In [0]:
# Funcion para leer una imagen
def read_image(row):
  path = os.path.join(images_path, row['dir'], row['image'])
  raw = cv2.imread(path,0)
  proc = None
  if raw is not None: # TODO: algunas imagenes no existen
    raw = cv2.resize(raw, dsize=(image_shape[0], image_shape[1])) # TODO: tendremos que redimensionar los puntos tambien
    proc = cv2.normalize(raw, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
  return raw, proc

# Funcion para leer las coordenadas
def read_label(row, max_points = 1):
  coords_split = row['coords'].split('),(')
  coords_split = list(map(lambda x : x.replace('(','').replace(')',''), coords_split))
  row_coords = []
  if len(coords_split) > max_points: # TODO: Que hacemos con estos puntos?
    return None
  for coords in coords_split:
    numbers = coords.split(',')
    numbers = list(map(lambda x : int(x), numbers))
    row_coords.extend(numbers)
  return row_coords

# Funcion que devuelve las imagenes y las coordenadas
def read_images(data):
  raws = []
  procs = []
  labels = []
  for index, row in data.iterrows():
    raw, proc = read_image(row)
    label = read_label(row)
    if raw is not None and label is not None:
      raws.append(raw)
      procs.append(proc)
      labels.append(label)
  raws = np.expand_dims(np.array(raws), axis=-1)
  procs = np.expand_dims(np.array(procs), axis=-1)
  labels = np.array(labels)
  return raws, procs, labels

In [0]:
# Generamos las imagenes y los outputs
raws, procs, labels = read_images(data)
print(raws.shape)
print(procs.shape)
print(labels.shape)

In [0]:
# Mostramos una imagen con el punto de la figura marcado
def show_image(image, point=None):
  if point is not None:
    image_with_circle = cv2.circle(image, (point[0], point[1]), 3, (0, 0, 0), -1)
  cv2_imshow(image_with_circle)

show_image(raws[50], labels[50])

In [0]:
# Creamos el modelo
def create_model():
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=image_shape))
  model.add(tf.keras.layers.MaxPooling2D((2, 2)))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(2, activation='relu'))
  return model

# Creamos los datasets de train y test
def generate_datasets(images, labels):
  return train_test_split(images, labels, test_size=test_dataset_size)

In [0]:
model = create_model()
train_images, test_images, train_labels, test_labels = generate_datasets(procs, labels)

In [0]:
# Compilamos y entrenamos el modelo
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
history = model.fit(train_images, train_labels, validation_data=(test_images, test_labels), epochs=10)